# Distributed model inference using TensorFlow Keras
From: https://docs.databricks.com/_static/notebooks/deep-learning/keras-metadata.html

In [1]:
import os
import shutil
import subprocess
import time
import pandas as pd
from PIL import Image
import numpy as np
import uuid
 
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
 
from pyspark.sql.functions import col, pandas_udf, PandasUDFType

In [2]:
file_name = "image_data.parquet"
output_file_path = "predictions"

### Prepare trained model and data for inference

Load the ResNet-50 Model and broadcast the weights.

In [3]:
model = ResNet50()
bc_model_weights = sc.broadcast(model.get_weights())

Load the data and save the datasets to one Parquet file.

In [4]:
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(origin=dataset_url,
                                   fname='flower_photos',
                                   untar=True)
data_dir = pathlib.Path(data_dir)

In [5]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

3670


In [6]:
import os
files = [os.path.join(dp, f) for dp, dn, filenames in os.walk(data_dir) for f in filenames if os.path.splitext(f)[1] == '.jpg']
files = files[:2048]
len(files)

2048

In [7]:
print(data_dir)

/home/leey/.keras/datasets/flower_photos


In [8]:
image_data = []
for file in files:
    img = Image.open(file)
    img = img.resize([224, 224])
    data = np.asarray(img, dtype="float32").reshape([224*224*3])

    image_data.append({"data": data})

pandas_df = pd.DataFrame(image_data, columns=['data'])
pandas_df.to_parquet(file_name)
# os.makedirs(dbfs_file_path)
# shutil.copyfile(file_name, dbfs_file_path+file_name)

### Save Model


In [9]:
subprocess.call("rm -rf resnet50_model".split())

0

In [10]:
model.save('resnet50_model')

INFO:tensorflow:Assets written to: resnet50_model/assets


INFO:tensorflow:Assets written to: resnet50_model/assets


### Load the data into Spark DataFrames

In [11]:
from pyspark.sql.types import *
df = spark.read.parquet(file_name)
print(df.count())

2048


In [12]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")
spark.conf.set("spark.sql.parquet.columnarReaderBatchSize", "1024")

In [13]:
assert len(df.head()) > 0, "`df` should not be empty" # This line will fail if the vectorized reader runs out of memory

### Model inference via pandas UDF

In [14]:
def parse_image(image_data):
    image = tf.image.convert_image_dtype(
        image_data, dtype=tf.float32) * (2. / 255) - 1
    image = tf.reshape(image, [224, 224, 3])
    return image

In [15]:
@pandas_udf(ArrayType(FloatType()), PandasUDFType.SCALAR_ITER)
def predict_batch_udf(image_batch_iter):
    batch_size = 64
    model = ResNet50(weights=None)
    model.set_weights(bc_model_weights.value)
    for image_batch in image_batch_iter:
        images = np.vstack(image_batch)
        dataset = tf.data.Dataset.from_tensor_slices(images)
        dataset = dataset.map(parse_image, num_parallel_calls=8).prefetch(
            5000).batch(batch_size)
        preds = model.predict(dataset)
        yield pd.Series(list(preds))

/home/leey/devpub/spark/python/pyspark/sql/pandas/functions.py:399: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


In [16]:
%%time
predictions_df = df.select(predict_batch_udf(col("data")).alias("prediction"))
predictions_df.show(truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                              prediction|
+------------------------------------------------------------------------------------------------------------------------+
|[1.7195931E-4, 3.2735552E-4, 7.533328E-5, 1.9810574E-4, 6.6188135E-5, 4.304618E-4, 1.3097588E-5, 5.2791635E-5, 2.3571...|
|[7.738322E-5, 5.935413E-4, 1.3075814E-4, 1.8000253E-4, 1.3001164E-4, 2.2790757E-4, 5.0780895E-5, 2.3768713E-4, 3.8676...|
|[2.3994662E-5, 3.4409956E-4, 1.8440963E-4, 1.6311614E-4, 1.4301096E-4, 8.883273E-5, 2.6071444E-5, 2.3609246E-4, 1.481...|
|[7.33013E-5, 2.6747186E-4, 1.2007599E-4, 2.0717822E-4, 8.630799E-5, 2.2318888E-4, 1.3459768E-5, 8.564859E-5, 1.482114...|
|[5.592278E-5, 2.5165555E-4, 1.3106635E-4, 1.5746983E-4, 8.4952095E-5, 2.1269226E-4, 9.83865E-6, 1.6246884E-4, 1.24666...|
|[1.1709497E-4, 

In [17]:
%%time
predictions_df.write.mode("overwrite").parquet(output_file_path)

CPU times: user 26 ms, sys: 8.71 ms, total: 34.7 ms
Wall time: 49.9 s


### Model inference using Spark DL API

In [18]:
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.functions import struct, col
from pyspark.sql.types import ArrayType, FloatType

In [19]:
def predict_batch_fn():
    import tensorflow as tf
    from tensorflow.keras.applications.resnet50 import ResNet50
    model = ResNet50()
    def predict(inputs):
        inputs = inputs * (2. / 255) - 1
        return model.predict(inputs)
    return predict

In [20]:
classify = predict_batch_udf(predict_batch_fn,
                             input_tensor_shapes=[[224, 224, 3]],
                             return_type=ArrayType(FloatType()),
                             batch_size=50)

In [21]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")
spark.conf.set("spark.sql.parquet.columnarReaderBatchSize", "1024")

In [22]:
df = spark.read.parquet("image_data.parquet")

In [23]:
%%time
# first pass caches model/fn
predictions = df.select(classify(struct("data")).alias("prediction"))
predictions.show(truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                              prediction|
+------------------------------------------------------------------------------------------------------------------------+
|[1.7195931E-4, 3.2735552E-4, 7.533328E-5, 1.9810574E-4, 6.6188135E-5, 4.304618E-4, 1.3097588E-5, 5.2791635E-5, 2.3571...|
|[7.738322E-5, 5.935413E-4, 1.3075814E-4, 1.8000253E-4, 1.3001164E-4, 2.2790757E-4, 5.0780895E-5, 2.3768713E-4, 3.8676...|
|[2.3994662E-5, 3.4409956E-4, 1.8440963E-4, 1.6311614E-4, 1.4301096E-4, 8.883273E-5, 2.6071444E-5, 2.3609246E-4, 1.481...|
|[7.33013E-5, 2.6747186E-4, 1.2007599E-4, 2.0717822E-4, 8.630799E-5, 2.2318888E-4, 1.3459768E-5, 8.564859E-5, 1.482114...|
|[5.592278E-5, 2.5165555E-4, 1.3106635E-4, 1.5746983E-4, 8.4952095E-5, 2.1269226E-4, 9.83865E-6, 1.6246884E-4, 1.24666...|
|[1.1709497E-4, 

In [24]:
%%time
predictions = df.select(classify("data").alias("prediction"))
predictions.show(truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                              prediction|
+------------------------------------------------------------------------------------------------------------------------+
|[1.7195931E-4, 3.2735552E-4, 7.533328E-5, 1.9810574E-4, 6.6188135E-5, 4.304618E-4, 1.3097588E-5, 5.2791635E-5, 2.3571...|
|[7.738322E-5, 5.935413E-4, 1.3075814E-4, 1.8000253E-4, 1.3001164E-4, 2.2790757E-4, 5.0780895E-5, 2.3768713E-4, 3.8676...|
|[2.3994662E-5, 3.4409956E-4, 1.8440963E-4, 1.6311614E-4, 1.4301096E-4, 8.883273E-5, 2.6071444E-5, 2.3609246E-4, 1.481...|
|[7.33013E-5, 2.6747186E-4, 1.2007599E-4, 2.0717822E-4, 8.630799E-5, 2.2318888E-4, 1.3459768E-5, 8.564859E-5, 1.482114...|
|[5.592278E-5, 2.5165555E-4, 1.3106635E-4, 1.5746983E-4, 8.4952095E-5, 2.1269226E-4, 9.83865E-6, 1.6246884E-4, 1.24666...|
|[1.1709497E-4, 

In [25]:
%%time
predictions = df.select(classify(col("data")).alias("prediction"))
predictions.write.mode("overwrite").parquet(output_file_path + "_1")

CPU times: user 30.8 ms, sys: 4.88 ms, total: 35.7 ms
Wall time: 49.1 s


### Using Triton Inference Server

Note: you can restart the kernel and run from this point to simulate running in a different node or environment.

In [26]:
import os
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.functions import col, struct
from pyspark.sql.types import ArrayType, FloatType

In [27]:
%%bash
# copy model to expected layout for Triton
rm -rf models
mkdir -p models/resnet50/1
cp -r resnet50_model models/resnet50/1/model.savedmodel

# add config.pbtxt
cp models_config/resnet50/config.pbtxt models/resnet50/config.pbtxt

#### Start Triton Server on each executor

In [28]:
num_executors = 1
triton_models_dir = "{}/models".format(os.getcwd())
nodeRDD = sc.parallelize(list(range(num_executors)), num_executors)

def start_triton(it):
    import docker
    import time
    import tritonclient.grpc as grpcclient
    
    client=docker.from_env()
    containers=client.containers.list(filters={"name": "spark-triton"})
    if containers:
        print(">>>> containers: {}".format([c.short_id for c in containers]))
    else:
        container=client.containers.run(
            "nvcr.io/nvidia/tritonserver:23.04-py3", "tritonserver --model-repository=/models",
            detach=True,
            device_requests=[docker.types.DeviceRequest(device_ids=["0"], capabilities=[['gpu']])],
            name="spark-triton",
            network_mode="host",
            remove=True,
            shm_size="512M",
            volumes={triton_models_dir: {"bind": "/models", "mode": "ro"}}
        )
        print(">>>> starting triton: {}".format(container.short_id))

        # wait for triton to be running
        time.sleep(15)
        client = grpcclient.InferenceServerClient("localhost:8001")
        ready = False
        while not ready:
            try:
                ready = client.is_server_ready()
            except Exception as e:
                time.sleep(5)

    return [True]

nodeRDD.mapPartitions(start_triton).collect()

[True]

#### Run inference

In [29]:
def triton_fn(triton_uri, model_name):
    import numpy as np
    import tritonclient.grpc as grpcclient
    
    np_types = {
      "BOOL": np.dtype(np.bool8),
      "INT8": np.dtype(np.int8),
      "INT16": np.dtype(np.int16),
      "INT32": np.dtype(np.int32),
      "INT64": np.dtype(np.int64),
      "FP16": np.dtype(np.float16),
      "FP32": np.dtype(np.float32),
      "FP64": np.dtype(np.float64),
      "FP64": np.dtype(np.double),
      "BYTES": np.dtype(object)
    }

    client = grpcclient.InferenceServerClient(triton_uri)
    model_meta = client.get_model_metadata(model_name)
    
    def predict(inputs):
        if isinstance(inputs, np.ndarray):
            # single ndarray input
            inputs = inputs * (2. / 255) - 1  # add normalization
            request = [grpcclient.InferInput(model_meta.inputs[0].name, inputs.shape, model_meta.inputs[0].datatype)]
            request[0].set_data_from_numpy(inputs.astype(np_types[model_meta.inputs[0].datatype]))
        else:
            # dict of multiple ndarray inputs
            request = [grpcclient.InferInput(i.name, inputs[i.name].shape, i.datatype) for i in model_meta.inputs]
            for i in request:
                i.set_data_from_numpy(inputs[i.name()].astype(np_types[i.datatype()]))
        
        response = client.infer(model_name, inputs=request)
        
        if len(model_meta.outputs) > 1:
            # return dictionary of numpy arrays
            return {o.name: response.as_numpy(o.name) for o in model_meta.outputs}
        else:
            # return single numpy array
            return response.as_numpy(model_meta.outputs[0].name)
        
    return predict

In [30]:
from functools import partial

classify = predict_batch_udf(partial(triton_fn, triton_uri="localhost:8001", model_name="resnet50"),
                             input_tensor_shapes=[[224, 224, 3]],
                             return_type=ArrayType(FloatType()),
                             batch_size=50)

In [31]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")
spark.conf.set("spark.sql.parquet.columnarReaderBatchSize", "1024")

In [32]:
df = spark.read.parquet("image_data.parquet")

In [33]:
%%time
# first pass caches model/fn
predictions = df.select(classify(struct("data")).alias("prediction"))
predictions.show(truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                              prediction|
+------------------------------------------------------------------------------------------------------------------------+
|[1.7042673E-4, 3.267522E-4, 7.581915E-5, 1.9806583E-4, 6.629557E-5, 4.2843417E-4, 1.31030665E-5, 5.2828378E-5, 2.3425...|
|[7.658893E-5, 5.898446E-4, 1.3134143E-4, 1.7956285E-4, 1.2959713E-4, 2.2623697E-4, 5.0602717E-5, 2.387755E-4, 3.85566...|
|[2.3643051E-5, 3.3990925E-4, 1.8499317E-4, 1.6320233E-4, 1.4257104E-4, 8.804509E-5, 2.6084534E-5, 2.3692146E-4, 1.479...|
|[7.265595E-5, 2.660495E-4, 1.2043192E-4, 2.0687138E-4, 8.6514396E-5, 2.2079627E-4, 1.3474356E-5, 8.564823E-5, 1.47982...|
|[5.583156E-5, 2.5246604E-4, 1.3152408E-4, 1.5752943E-4, 8.486259E-5, 2.1114835E-4, 9.902404E-6, 1.6352077E-4, 1.25058...|
|[1.1620977E-4, 

In [34]:
%%time
predictions = df.select(classify("data").alias("prediction"))
predictions.show(truncate=120)

+------------------------------------------------------------------------------------------------------------------------+
|                                                                                                              prediction|
+------------------------------------------------------------------------------------------------------------------------+
|[1.7042673E-4, 3.267522E-4, 7.581915E-5, 1.9806583E-4, 6.629557E-5, 4.2843417E-4, 1.31030665E-5, 5.2828378E-5, 2.3425...|
|[7.658893E-5, 5.898446E-4, 1.3134143E-4, 1.7956285E-4, 1.2959713E-4, 2.2623697E-4, 5.0602717E-5, 2.387755E-4, 3.85566...|
|[2.3643051E-5, 3.3990925E-4, 1.8499317E-4, 1.6320233E-4, 1.4257104E-4, 8.804509E-5, 2.6084534E-5, 2.3692146E-4, 1.479...|
|[7.265595E-5, 2.660495E-4, 1.2043192E-4, 2.0687138E-4, 8.6514396E-5, 2.2079627E-4, 1.3474356E-5, 8.564823E-5, 1.47982...|
|[5.583156E-5, 2.5246604E-4, 1.3152408E-4, 1.5752943E-4, 8.486259E-5, 2.1114835E-4, 9.902404E-6, 1.6352077E-4, 1.25058...|
|[1.1620977E-4, 

In [35]:
%%time
predictions = df.select(classify(col("data")).alias("prediction"))
predictions.write.mode("overwrite").parquet(output_file_path + "_2")

CPU times: user 18.8 ms, sys: 6.13 ms, total: 25 ms
Wall time: 22.2 s


#### Stop Triton Server on each executor

In [36]:
def stop_triton(it):
    import docker
    import time
    
    client=docker.from_env()
    containers=client.containers.list(filters={"name": "spark-triton"})
    print(">>>> stopping containers: {}".format([c.short_id for c in containers]))
    if containers:
        container=containers[0]
        container.stop(timeout=120)

    return [True]

nodeRDD.mapPartitions(stop_triton).collect()

[True]

In [37]:
spark.stop()